In [1]:
! pip install apyori

Looking in indexes: https://pypi.doubanio.com/simple
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for apyori: filename=apyori-1.1.2-py3-none-any.whl size=5975 sha256=4427739ea95401127844f5dabaa8e82b4ba6fe6b98b8861d84f33fe2cc0e8de4
  Stored in directory: c:\users\zhesiwode\appdata\local\pip\cache\wheels\76\27\56\512c1c65ec74c98fea782b2632f07b9d8be16a82611af10af6
Successfully built apyori


In [2]:
import pandas
movie = pandas.read_csv('Data/movies.csv')
movie.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [7]:
movie_dic = {}
for rec in movie.iterrows():
    movie_dic[rec[1].movieId]  = rec[1].title

In [9]:
movie_dic.get(2)

'Jumanji (1995)'

In [10]:
import pandas
import datetime

df = pandas.read_csv('Data/ratings.csv')
df.info()
#df = df[df['timestamp'] >= 1325376000]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100004 entries, 0 to 100003
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100004 non-null  int64  
 1   movieId    100004 non-null  int64  
 2   rating     100004 non-null  float64
 3   timestamp  100004 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [11]:
df = df[df['timestamp'] >= 1325376000]

In [12]:
df.head()

,userId,movieId,rating,timestamp
790,11,50,5.0,1391658537
791,11,70,1.0,1391656827
792,11,126,4.0,1391657561
793,11,169,3.0,1391657297
794,11,296,5.0,1391658423


In [13]:
from apyori import apriori
transactions = [ele for ele in df.groupby('userId')['movieId'].apply(list)]
rules = apriori(transactions, min_support = 0.2, min_confidence = 0.5, min_lift = 3, min_length = 2)

results = list(rules)

In [27]:
### 取出有关联的几个数据
for rec in results:
    left_hands = rec.ordered_statistics[0].items_base
    right_hands = rec.ordered_statistics[0].items_add
#     print(left_hands)
#     print([item for item in left_hands])
    l = ';'.join([movie_dic.get(item) for item in left_hands])
    r = ';'.join([movie_dic.get(item) for item in right_hands])
    print('{} => {}\n'.format(l,r))
        
    #print(';'.join([movie_dic.get(item) for item in rec.items]))

Kill Bill: Vol. 1 (2003) => Kill Bill: Vol. 2 (2004)

Lord of the Rings: The Two Towers, The (2002);Star Wars: Episode IV - A New Hope (1977) => Lord of the Rings: The Return of the King, The (2003);Star Wars: Episode VI - Return of the Jedi (1983)

Matrix, The (1999);Star Wars: Episode V - The Empire Strikes Back (1980) => Star Wars: Episode VI - Return of the Jedi (1983);Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)

Lord of the Rings: The Two Towers, The (2002);Star Wars: Episode V - The Empire Strikes Back (1980) => Lord of the Rings: The Return of the King, The (2003);Star Wars: Episode VI - Return of the Jedi (1983)

Batman Begins (2005);Matrix, The (1999) => Lord of the Rings: The Two Towers, The (2002);Inception (2010)

Lord of the Rings: The Fellowship of the Ring, The (2001);Batman Begins (2005) => Lord of the Rings: The Two Towers, The (2002);Inception (2010)

Lord of the Rings: The Fellowship of the Ring, The (2001);Dark Knight, The (2008) =

In [ ]:
### apriori的输入值必须是数组
### 每一行代表一个事物（可能是一个人），列代表它所做过的事（可以通过groupby得到）
### 判断某人在做了这件事后，做另一件事的几率是怎么样的